In [3]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [4]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/data/'
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"
data_time = '20180920'

In [5]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [6]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [9]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA):
    if file.endswith("art.csv.gz"):
        UKWIKI_ART_FNMS.append(file)
UKWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA):
    if file.endswith("red.csv.gz"):
        UKWIKI_RED_FNMS.append(file)   

In [11]:
print('UKWIKI_ART_FNMS:', UKWIKI_ART_FNMS)
print('UKWIKI_RED_FNMS:', UKWIKI_RED_FNMS)

UKWIKI_ART_FNMS: ['enwiki-20180920-pages-meta-current1.xml-p10p30303.bz2_art.csv.gz']
UKWIKI_RED_FNMS: ['enwiki-20180920-pages-meta-current1.xml-p10p30303.bz2_red.csv.gz']


In [12]:
df_uk_id_name = None
for fn in UKWIKI_ART_FNMS:
    fn = PATH_TO_DATA+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    print(df_articles.head(100))
    df_id_name_tmp = df_articles[['id', 'title', 'text_len']].drop_duplicates()
    df_id_name_tmp.columns = ['id', 'title', 'length']
    #print("df_id_name_tmp size: {}".format(df_id_name_tmp.shape))
    if df_uk_id_name is not None:
        df_uk_id_name = df_uk_id_name.append(df_id_name_tmp)        
        #print("append")
    else:
        df_uk_id_name = df_id_name_tmp
        #print("assign")
    print("df_uk_id_name size: {}".format(df_uk_id_name.shape))
    os.remove(fn_new)
df_uk_id_name = df_uk_id_name.drop_duplicates()

/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/enwiki-20180920-pages-meta-current1.xml-p10p30303.bz2_art.csv.gz
    id      title  text_len  link_pos  \
0   12  Anarchism    193812      2413   
1   12  Anarchism    193812      3270   
2   12  Anarchism    193812      3777   
3   12  Anarchism    193812      6900   
4   12  Anarchism    193812      6992   
5   12  Anarchism    193812      7022   
6   12  Anarchism    193812      7151   
7   12  Anarchism    193812      7183   
8   12  Anarchism    193812      8791   
9   12  Anarchism    193812      8850   
10  12  Anarchism    193812      8885   
11  12  Anarchism    193812      9171   
12  12  Anarchism    193812      9194   
13  12  Anarchism    193812      9799   
14  12  Anarchism    193812     10263   
15  12  Anarchism    193812     10385   
16  12  Anarchism    193812     11169   
17  12  Anarchism    193812     11360   
18  12  Anarchism    193812     11524   
19  12  Anarchism  

In [8]:
df_uk_id_name.to_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", 
                     index = False, encoding='UTF-8', quotechar="\"", sep ="^")

In [9]:
df_uk_id_name = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", encoding='utf-8', sep="^")
df_uk_id_name = df_uk_id_name[['id', 'title']]
df_uk_id_name.head()

,id,title
0,3,Головна сторінка
1,13,Географія
2,584,Атом
3,585,Мільярд
4,586,Ядро


In [10]:
df_uk_id_orig_title_alias = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_orig_title_alias.csv",  encoding='UTF-8', quotechar="\"")
df_uk_id_orig_title_alias.head()

,id_orig,title_alias
0,610,Esperanto
1,9701,Wikipedia
2,3,HomePage
3,61817,Капустянко Микола
4,6083,Володимир Винниченко


In [11]:
df_uk_id_orig_title_alias.columns = ['id', 'title']
df_uk_id_name_all = df_uk_id_name.append(df_uk_id_orig_title_alias)
df_uk_id_name_all.shape

(1272761, 2)

In [12]:
df_uk_id_name_all['title'] = df_uk_id_name_all['title'].str.strip()

In [13]:
df_uk_id_name_all['title_lc'] = df_uk_id_name_all['title'].str.lower()
df_uk_id_name_all = df_uk_id_name_all[['id', 'title_lc']]
df_uk_id_name_all = df_uk_id_name_all.drop_duplicates()
df_uk_id_name_all.shape

(1264552, 2)

In [14]:
df_uk_id_name_all.head()

,id,title_lc
0,3,головна сторінка
1,13,географія
2,584,атом
3,585,мільярд
4,586,ядро


In [13]:
fn = PATH_TO_DATA+UKWIKI_ART_FNMS[0]
print(fn)
fn_new = unpack(fn)
df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
df_articles['link_val'] = df_articles['link_val'].str.split("#").str[0] # ???
df_articles = df_articles[df_articles['link_val'].str.strip() != '']
df_articles['link_pos_perc'] = (df_articles['link_pos']/df_articles['text_len']).round(3)    
df_articles['link_val_lc'] = df_articles['link_val'].str.lower()
df_articles.head(100)

/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/enwiki_20180920/enwiki-20180920-pages-meta-current1.xml-p10p30303.bz2_art.csv.gz


,id,title,text_len,link_pos,link_val,link_txt,link_pos_perc,link_val_lc
0,12,Anarchism,193812,2413,State (polity),state,0.012,state (polity)
1,12,Anarchism,193812,3270,The Globe and Mail,NaN,0.017,the globe and mail
2,12,Anarchism,193812,3777,Routledge Encyclopedia of Philosophy,NaN,0.019,routledge encyclopedia of philosophy
3,12,Anarchism,193812,6900,Anarchist schools of thought,NaN,0.036,anarchist schools of thought
4,12,Anarchism,193812,6992,individualism,NaN,0.036,individualism
5,12,Anarchism,193812,7022,collectivism,NaN,0.036,collectivism
6,12,Anarchism,193812,7151,Social anarchism,social,0.037,social anarchism
7,12,Anarchism,193812,7183,individualist anarchism,NaN,0.037,individualist anarchism
8,12,Anarchism,193812,8791,French Revolution,NaN,0.045,french revolution
9,12,Anarchism,193812,8850,Maximilien Robespierre,NaN,0.046,maximilien robespierre


In [17]:
df_articles[df_articles['link_val_lc'] == 'премія «золотий глобус» за найкращий  фільм — драма']

,id,title,text_len,link_pos,link_val,link_txt,link_pos_perc,link_val_lc
15882153,2101563,Кардинал (фільм),9436,1489,Премія «Золотий глобус» за найкращий фільм — ...,найкращий драматичний фільм,0.158,премія «золотий глобус» за найкращий фільм — ...


In [20]:
df_uk_titles_series = df_articles.iloc[:,7]

In [21]:
df_uk_titles_series_stripped = df_uk_titles_series.str.replace('  ', ' ')

In [23]:
df_titles_stripped = df_uk_titles_series_stripped.to_frame()

In [29]:
#replace not stripped in df_articles with df_titles_stripped
df_articles_less = df_articles[['id', 'title', 'link_val']]

In [31]:
df_articles = pd.concat([df_articles_less, df_titles_stripped], axis=1)

In [32]:
df_articles['link_val_lc'] = df_articles['link_val_lc'].str.strip()

In [33]:
df_articles = pd.merge(df_articles, df_uk_id_name_all, how='left', left_on=['link_val_lc'], right_on=['title_lc'])

In [34]:
df_articles.head()

,id_x,title,link_val,link_val_lc,id_y,title_lc
0,3,Головна сторінка,NaN,NaN,190261.0,NaN
1,3,Головна сторінка,NaN,NaN,937102.0,NaN
2,3,Головна сторінка,NaN,NaN,1719596.0,NaN
3,13,Географія,Об'єкт (філософія),об'єкт (філософія),884280.0,об'єкт (філософія)
4,13,Географія,Ератосфен,ератосфен,21623.0,ератосфен


In [35]:
df_articles = df_articles[['id_x', 'id_y', 'link_val']]
df_articles.columns = ['id', 'link_id', 'link_val']
df_articles.head()

,id,link_id,link_val
0,3,190261.0,NaN
1,3,937102.0,NaN
2,3,1719596.0,NaN
3,13,884280.0,Об'єкт (філософія)
4,13,21623.0,Ератосфен


In [37]:
df_articles = df_articles[pd.notnull(df_articles['link_val'])]

In [38]:
df_articles[['link_id']] = df_articles[['link_id']].fillna(-1).astype(int)
df_articles['is_red_link'] = np.where(df_articles['link_id'] == -1, True, False)

In [40]:
df_articles.to_csv('ukwiki-20180920-pages-links.csv.gz', index = False, encoding='UTF-8', quotechar="\"")

In [ ]:
pack_and_remove(fn_new)

In [41]:
!pwd

/media/andrii/earth/Katia/BabelNet_baseline/uk-en_redlinks/code/get_redlinks_from_dumps


In [6]:
df_id_name = pd.read_csv(PATH_TO_DATA+'enwiki-' + data_time + '-id_name.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")

In [21]:
df_id_name.head()

,id,title
0,12,Anarchism
1,25,Autism
2,39,Albedo
3,290,A
4,303,Alabama


In [18]:
fn = PATH_TO_DATA+"ukwiki-" + data_time + "-langlinks_en.csv"
df_en_langlinks = pd.read_csv(fn, encoding='UTF-16', sep="^")

In [27]:
df_en_langlinks = df_en_langlinks[pd.notnull(df_en_langlinks['ll_title'])]

In [28]:
df_en_langlinks.head()

,ll_from,ll_lang,ll_title
23,2112530,en,! (The Dismemberment Plan album)
24,1366003,en,! (disambiguation)
25,2141483,en,!!
26,425480,en,!!!
27,848226,en,!Action Pact!


In [23]:
#df_uk_en_links = pd.merge(df_en_langlinks, df_id_name, how='left', left_on=['ll_title'], right_on=['title'])

In [ ]:
df_uk_en_links = df_uk_en_links[['ll_from', 'id']]
df_uk_en_links.columns = ['id_uk', 'id_en']
df_uk_en_links[['id_uk']] = df_uk_en_links[['id_uk']].fillna(-1).astype(int)
df_uk_en_links[['id_en']] = df_uk_en_links[['id_en']].fillna(-1).astype(int)
df_uk_en_links.to_csv(PATH_TO_DATA_UK + data_time + "-langlinks_uk_en.csv", index = False, 
                      encoding='ISO-8859-1', quotechar="'", 
                      escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)
pack_and_remove(fn_new)

In [30]:
df_en_langlinks.to_csv(PATH_TO_DATA + data_time + "-langlinks_uk_en.csv", index = False, 
                      encoding='ISO-8859-1', quotechar="'", 
                      escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)